# 第4章 朴素贝叶斯

1．朴素贝叶斯法是典型的生成学习方法。生成方法由训练数据学习联合概率分布
$P(X,Y)$，然后求得后验概率分布$P(Y|X)$。具体来说，利用训练数据学习$P(X|Y)$和$P(Y)$的估计，得到联合概率分布：

$$P(X,Y)＝P(Y)P(X|Y)$$

概率估计方法可以是极大似然估计或贝叶斯估计。

2．朴素贝叶斯法的基本假设是条件独立性，

$$\begin{aligned} P(X&=x | Y=c_{k} )=P\left(X^{(1)}=x^{(1)}, \cdots, X^{(n)}=x^{(n)} | Y=c_{k}\right) \\ &=\prod_{j=1}^{n} P\left(X^{(j)}=x^{(j)} | Y=c_{k}\right) \end{aligned}$$


这是一个较强的假设。由于这一假设，模型包含的条件概率的数量大为减少，朴素贝叶斯法的学习与预测大为简化。因而朴素贝叶斯法高效，且易于实现。其缺点是分类的性能不一定很高。

3．朴素贝叶斯法利用贝叶斯定理与学到的联合概率模型进行分类预测。

$$P(Y | X)=\frac{P(X, Y)}{P(X)}=\frac{P(Y) P(X | Y)}{\sum_{Y} P(Y) P(X | Y)}$$
 
将输入$x$分到后验概率最大的类$y$。

$$y=\arg \max _{c_{k}} P\left(Y=c_{k}\right) \prod_{j=1}^{n} P\left(X_{j}=x^{(j)} | Y=c_{k}\right)$$

后验概率最大等价于0-1损失函数时的期望风险最小化。


模型：

- 高斯模型
- 多项式模型
- 伯努利模型

In [7]:
import numpy as np                  # numpy:Python中基于数组对象的科学计算库，支持大量的维度数组与矩阵运算，以及大量的数学函数库。
import pandas as pd                 # 基于numpy,用于处理文本或者表格数据
import matplotlib.pyplot as plt     # 绘图函数
%matplotlib inline

from sklearn.datasets import load_iris                  # 鸢尾花数据集,包含了3类鸢尾花的4个特征（萼片长度、萼片宽度、花瓣长度、花瓣宽度），共有150个样本。
from sklearn.model_selection import train_test_split    # 导入数据切分函数--train_test_split; 其输入为待切分的特征X 和 相应变量y。输出按顺序为：切分后的训练集特征X_train，测试集特征X_test，训练集响应变量y_train，测试集响应变量y_test。

from collections import Counter     # 导入counter()函数，可以用来统计一个 python 列表、字符串、元组等可迭代对象中每个元素出现的次数，并返回一个字典。
import math                         # 导入数学库

In [8]:
# data
def create_data():  # 创建数据
    iris = load_iris()              # 加载鸢尾花数据集
    
    # 创建一个 DataFrame 对象：二维表格型数据，含有一组有序的列，每列可以是不同的值类型。既有行索引，也有列索引。
    df = pd.DataFrame(iris.data, columns=iris.feature_names)    # 将iris.data作为数据源，iris.feature_name(特征名称 每列的属性名，['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)'] ）作为列名。
    df['label'] = iris.target       # iris.target:目标属性，品种信息；将此值赋值给df的lable
    df.columns = [
        'sepal length', 'sepal width', 'petal length', 'petal width', 'label'
    ]
    data = np.array(df.iloc[:100, :])   # 读取前100行的数据并转化为array格式
    # print(data)
    return data[:, :-1], data[:, -1]    # 将除最后一列和最后一列分别返回

In [9]:
X, y = create_data()            # 加载数据集；选取除了最后一列之外的所有列变量作为X变量；选取最后一列列变量作为y变量
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)    # train_test_split() 其输入为待切分的特征X 和 相应变量y。输出按顺序为：切分后的训练集特征X_train，测试集特征X_test，训练集响应变量y_train，测试集响应变量y_test。

In [10]:
X_test[0], y_test[0]    # 打印测试集第一行数据

(array([5.5, 2.6, 4.4, 1.2]), 1.0)

参考：https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/

## GaussianNB 高斯朴素贝叶斯

特征的可能性被假设为高斯

概率密度函数：
$$P(x_i | y_k)=\frac{1}{\sqrt{2\pi\sigma^2_{yk}}}exp(-\frac{(x_i-\mu_{yk})^2}{2\sigma^2_{yk}})$$

数学期望(mean)：$\mu$

方差：$\sigma^2=\frac{\sum(X-\mu)^2}{N}$

In [57]:
class NaiveBayes:   # 朴素贝叶斯
    def __init__(self):
        self.model = None

    # 数学期望
    @staticmethod
    def mean(X):
        return sum(X) / float(len(X))

    # 标准差（方差）
    def stdev(self, X):
        avg = self.mean(X)  # 求期望
        return math.sqrt(sum([pow(x - avg, 2) for x in X]) / float(len(X)))

    # 概率密度函数
    def gaussian_probability(self, x, mean, stdev): # 求高斯概率密度函数
        exponent = math.exp(-(math.pow(x - mean, 2) /           # exp部分表达式
                              (2 * math.pow(stdev, 2))))
        return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent    # 拟合高斯概率密度函数

    # 处理X_train
    def summarize(self, train_data):        # 求每个元组的均值、标准差
        summaries = [(self.mean(i), self.stdev(i)) for i in zip(*train_data)]   # zip(*train_data) 对train_data数组进行解压（遍历每一行的每一个元素，并从数据集中返回一列数字）；并求期望和方差
        return summaries

    # 分类别求出数学期望和标准差
    def fit(self, X, y):
        labels = list(set(y))   # list(set())函数：对原列表去重并按从小到大排序
        data = {label: [] for label in labels}  # 创建字典data;得到y中的不同结果，eg:{0.0: [], 1.0: []}
        for f, label in zip(X, y):
            data[label].append(f)   # 将X的值添加lable所对应的key值下
        # 计算条件概率
        self.model = {
            label: self.summarize(value)    # 处理X_train;处理结果类似:{y的取值:[均值,标准差],[均值,标准差]}
            for label, value in data.items()    # 迭代data
        }
        return 'gaussianNB train done!'

    # 计算概率
    def calculate_probabilities(self, input_data):  # 计算概率
        probabilities = {}  # 创建空集合,用来保存类别标记和分到该类别的概率
        # self.model eg:{   0.0: [(4.982857142857142, 0.3376267713536559), (3.4657142857142866, 0.35931908393742834), (1.4799999999999998, 0.16527034130262364), (0.24000000000000002, 0.09319717960171478)], 
        #                   1.0: [(5.960000000000001, 0.43700277867701093), (2.8114285714285714, 0.2733615382826283), (4.362857142857144, 0.4154761222540536), (1.3628571428571434, 0.18835485398685153)]}
        for label, value in self.model.items(): # 迭代fit()方法中生成的model字典
            probabilities[label] = 1    # 赋初值为1；后续做乘法操作
            # 求input_data数组的第一个元素的高斯概率结果;再循环一次求input_data数组的下一个元素的高斯概率结果;依次循环.
            for i in range(len(value)): # 这里value是一个列表，每个元素为各个特征的（均值，标准差）
                mean, stdev = value[i]  # 取出期望和标准差
                probabilities[label] *= self.gaussian_probability(input_data[i], mean, stdev) # 高斯概率函数的结果相乘作为分到该lable的概率；gaussian_probability:计算高斯概率密度函数
        return probabilities    # 返回结果：eg:{0.0: 1.1712487714068567, 1.0: 1.4106989778814687e-23}

    # 类别
    def predict(self, X_test):  # X_test:[4.4,  3.2,  1.3,  0.2]
        label = sorted(                                     # 按概率值排序，并选择概率最大的类别
            self.calculate_probabilities(X_test).items(),   # 计算概率
            key=lambda x: x[-1])[-1][0]                     # 按照列表中最后一个元素排序
        return label

    def score(self, X_test, y_test):    # 计算预测的准确率
        right = 0
        for X, y in zip(X_test, y_test):
            label = self.predict(X) # 进行预测
            if label == y:          # 如果预测正确
                right += 1

        return right / float(len(X_test))   # 计算准确率并返回

In [55]:
model = NaiveBayes()    # 实例化模型

In [56]:
model.fit(X_train, y_train) # 使用训练数据进行训练

{0.0: [(4.982857142857142, 0.3376267713536559), (3.4657142857142866, 0.35931908393742834), (1.4799999999999998, 0.16527034130262364), (0.24000000000000002, 0.09319717960171478)], 1.0: [(5.960000000000001, 0.43700277867701093), (2.8114285714285714, 0.2733615382826283), (4.362857142857144, 0.4154761222540536), (1.3628571428571434, 0.18835485398685153)]}


'gaussianNB train done!'

In [53]:
print(model.predict([4.4,  3.2,  1.3,  0.2]))   # 预测分类

dict_items([(0.0, 1.1712487714068567), (1.0, 1.4106989778814687e-23)])
0.0
0.0


In [18]:
model.score(X_test, y_test) # 在测试集中计算预测的准确率

1.0

### scikit-learn实例

In [19]:
from sklearn.naive_bayes import GaussianNB  # 使用scikit 高斯朴素贝叶斯分类

In [20]:
clf = GaussianNB()              # 实例化模型
clf.fit(X_train, y_train)       # 使用训练数据进行训练

GaussianNB()

In [21]:
clf.score(X_test, y_test)       # 在测试集中计算预测的准确率

1.0

In [22]:
clf.predict([[4.4,  3.2,  1.3,  0.2]])  # 预测分类

array([0.])

In [23]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB # 伯努利模型和多项式模型

## 第4章朴素贝叶斯法-习题

### 习题4.1
&emsp;&emsp;用极大似然估计法推出朴素贝叶斯法中的概率估计公式(4.8)及公式 (4.9)。

**解答：**  
**第1步：**证明公式(4.8)：$\displaystyle P(Y=c_k) = \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k)}{N}$  
由于朴素贝叶斯法假设$Y$是定义在输出空间$\mathcal{Y}$上的随机变量，因此可以定义$P(Y=c_k)$概率为$p$。  
令$\displaystyle m=\sum_{i=1}^NI(y_i=c_k)$，得出似然函数：$$L(p)=f_D(y_1,y_2,\cdots,y_n|\theta)=\binom{N}{m}p^m(1-p)^{(N-m)}$$使用微分求极值，两边同时对$p$求微分：$$\begin{aligned}
0 &= \binom{N}{m}\left[mp^{(m-1)}(1-p)^{(N-m)}-(N-m)p^m(1-p)^{(N-m-1)}\right] \\
& = \binom{N}{m}\left[p^{(m-1)}(1-p)^{(N-m-1)}(m-Np)\right]
\end{aligned}$$可求解得到$\displaystyle p=0,p=1,p=\frac{m}{N}$  
显然$\displaystyle P(Y=c_k)=p=\frac{m}{N}=\frac{\displaystyle \sum_{i=1}^N I(y_i=c_k)}{N}$，公式(4.8)得证。

----

**第2步：**证明公式(4.9)：$\displaystyle P(X^{(j)}=a_{jl}|Y=c_k) = \frac{\displaystyle \sum_{i=1}^N I(x_i^{(j)}=a_{jl},y_i=c_k)}{\displaystyle \sum_{i=1}^N I(y_i=c_k)}$  
令$P(X^{(j)}=a_{jl}|Y=c_k)=p$，令$\displaystyle m=\sum_{i=1}^N I(y_i=c_k), q=\sum_{i=1}^N I(x_i^{(j)}=a_{jl},y_i=c_k)$，得出似然函数：$$L(p)=\binom{m}{q}p^q(i-p)^{m-q}$$使用微分求极值，两边同时对$p$求微分：$$\begin{aligned}
0 &= \binom{m}{q}\left[qp^{(q-1)}(1-p)^{(m-q)}-(m-q)p^q(1-p)^{(m-q-1)}\right] \\
& = \binom{m}{q}\left[p^{(q-1)}(1-p)^{(m-q-1)}(q-mp)\right]
\end{aligned}$$可求解得到$\displaystyle p=0,p=1,p=\frac{q}{m}$  
显然$\displaystyle P(X^{(j)}=a_{jl}|Y=c_k)=p=\frac{q}{m}=\frac{\displaystyle \sum_{i=1}^N I(x_i^{(j)}=a_{jl},y_i=c_k)}{\displaystyle \sum_{i=1}^N I(y_i=c_k)}$，公式(4.9)得证。

### 习题4.2
&emsp;&emsp;用贝叶斯估计法推出朴素贝叶斯法中的慨率估计公式(4.10)及公式(4.11)

**解答：**  
**第1步：**证明公式(4.11)：$\displaystyle P(Y=c_k) = \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k) + \lambda}{N+K \lambda}$  
加入先验概率，在没有任何信息的情况下，可以假设先验概率为均匀概率（即每个事件的概率是相同的）。  
可得$\displaystyle p=\frac{1}{K} \Leftrightarrow pK-1=0\quad(1)$  
根据习题4.1得出先验概率的极大似然估计是$\displaystyle pN - \sum_{i=1}^N I(y_i=c_k) = 0\quad(2)$  
存在参数$\lambda$使得$(1) \cdot \lambda + (2) = 0$  
所以有$$\lambda(pK-1) + pN - \sum_{i=1}^N I(y_i=c_k) = 0$$可得$\displaystyle P(Y=c_k) = \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k) + \lambda}{N+K \lambda}$，公式(4.11)得证。  

----

**第2步：**证明公式(4.10)：$\displaystyle P_{\lambda}(X^{(j)}=a_{jl} | Y = c_k) = \frac{\displaystyle \sum_{i=1}^N I(x_i^{(j)}=a_{jl},y_i=c_k) + \lambda}{\displaystyle \sum_{i=1}^N I(y_i=c_k) + S_j \lambda}$   
根据第1步，可同理得到$$
P(Y=c_k, x^{(j)}=a_{j l})=\frac{\displaystyle \sum_{i=1}^N I(y_i=c_k, x_i^{(j)}=a_{jl})+\lambda}{N+K S_j \lambda}$$  
$$\begin{aligned} 
P(x^{(j)}=a_{jl} | Y=c_k)
&= \frac{P(Y=c_k, x^{(j)}=a_{j l})}{P(y_i=c_k)} \\
&= \frac{\displaystyle \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k, x_i^{(j)}=a_{jl})+\lambda}{N+K S_j \lambda}}{\displaystyle \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k) + \lambda}{N+K \lambda}} \\
&= (\lambda可以任意取值，于是取\lambda = S_j \lambda) \\
&= \frac{\displaystyle \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k, x_i^{(j)}=a_{jl})+\lambda}{N+K S_j \lambda}}{\displaystyle \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k) + \lambda}{N+K S_j \lambda}} \\ 
&= \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k, x_i^{(j)}=a_{jl})+\lambda}{\displaystyle \sum_{i=1}^N I(y_i=c_k) + \lambda} (其中\lambda = S_j \lambda)\\
&= \frac{\displaystyle \sum_{i=1}^N I(x_i^{(j)}=a_{jl},y_i=c_k) + \lambda}{\displaystyle \sum_{i=1}^N I(y_i=c_k) + S_j \lambda}
\end{aligned} $$  
公式(4.11)得证。

----
参考代码：https://github.com/wzyonggege/statistical-learning-method

本文代码更新地址：https://github.com/fengdu78/lihang-code

习题解答：https://github.com/datawhalechina/statistical-learning-method-solutions-manual

中文注释制作：机器学习初学者公众号：ID:ai-start-com

配置环境：python 3.5+

代码全部测试通过。
![gongzhong](../gongzhong.jpg)